# **Simple CNN**

## **Imports**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

from utils import load_images
from utils import TrainingPlot
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.backend import clear_session
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from utils import load_images
from utils import TrainingPlot

clear_session()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.list_physical_devices())

ModuleNotFoundError: No module named 'seaborn'

## **Parameters**

In [3]:
MODEL_VERSION = "_cnn_1.1"

METADATA = pd.read_csv('../src/meta_data.csv')
IMAGE_FOLDER = "../src/final_data"

PARAMETERS = {
    "MODEL_VERSION": MODEL_VERSION,
    "IMG_HEIGHT": 128,
    "IMG_WIDTH": 128,
    "NUM_CLASSES": 19,
    "TEST_SPLIT": 0.2,
    "BATCH_SIZE": 128,
    "EPOCHS": 15,
    }

#with open(f"../src/models/resNet50/{MODEL_VERSION}/parameters{MODEL_VERSION}.json", "w") as json_file:
#    json.dump(PARAMETERS, json_file, indent=4)

## **Load Images**

In [4]:
with tf.device("CPU:0"):
    METADATA = pd.read_csv('../src/meta_data.csv')

    # Load images and labels
    image_folder = '../src/final_data'
    images, labels = load_images(image_folder, METADATA, PARAMETERS["IMG_HEIGHT"], PARAMETERS["IMG_WIDTH"])

    # Normalize images
    images = images / 255.0

    # Encode labels
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_categorical = to_categorical(labels_encoded)

    # Train / Test Split
    X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

    del images
    del labels
    del labels_categorical

Processing: 100%|██████████| 57141/57141 [08:47<00:00, 108.39it/s]


## **Create Model**

In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = (PARAMETERS["IMG_HEIGHT"], PARAMETERS["IMG_WIDTH"], 3)
num_classes = len(label_encoder.classes_)
model = create_cnn_model(input_shape, num_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## **Training**

In [ ]:
# Train the model
history = model.fit(X_train, y_train, 
                    epochs=12, 
                    validation_split=0.3, 
                    batch_size=32,
                    callbacks=[TrainingPlot(PARAMETERS["MODEL_VERSION"])])

In [ ]:
model.save(f"../src/models/model_v{PARAMETERS["MODEL_VERSION"]}.h5")
model.save(f"../src/models/model_v{PARAMETERS["MODEL_VERSION"]}.keras")

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

## **Test**

In [ ]:
with tf.device("CPU:0"):
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)

In [ ]:
with tf.device("CPU:0"):
    y_pred = model.predict(X_test)
    y_pred_class = np.argmax(y_pred, axis=1
    y_pred_class

In [ ]:
# convert predicted labels to string representation
y_pred_labels = [label_mapping[label] for label in y_pred_class]

# decode encoded labes to numeric values and convert the to the string representations again
y_test_numeric = np.argmax(y_test, axis=1)
y_test_labels = [label_mapping[label] for label in y_test_numeric]

In [ ]:
print('Classification Report:')
classification_report_str = classification_report(y_test_labels, y_pred_labels)
print(classification_report_str)

with open(f'../src/models/cnn/{PARAMETERS["MODEL_VERSION"]}/classification_report_{PARAMETERS["MODEL_VERSION"]}.txt', 'w') as f:
    f.write('Classification Report:\n')
    f.write(classification_report_str)
    f.write(f'\nAccuracy: {test_accuracy}\n')

print('Confusion Matrix:')
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)

plt.figure(figsize=(22, 16))
sns.heatmap(conf_matrix, annot=True, fmt='', cmap='Blues', 
            xticklabels=label_mapping.values(), yticklabels=label_mapping.values())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

plt.savefig(f"../src/models/cnn/{PARAMETERS["MODEL_VERSION"]}/conf_matrix_{PARAMETERS["MODEL_VERSION"]}.png", dpi=300)
plt.show()

In [ ]:
del X_train
del X_test
del y_train
del y_test
del images
del labels
del label_encoder
del labels_encoded
del model